In [1]:
## 2019/11/1 Hopfield Net trial 
%reload_ext autoreload
%autoreload 2
import matplotlib
matplotlib.use('qt5agg')

import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import random
import matplotlib.pyplot as plt
from ncautil.ncalearn import *
from ncautil.seqgen import SeqGen
from ncautil.seqmultip import *
from ncautil.seqmultip2 import *
from ncautil.datautil import *
from ncautil.ncamath import *
from ncautil.nlputil import NLPutil
import torch
import nltk
from tqdm import tqdm_notebook

In [3]:
nVcab=30000
nlp=NLPutil()
nlp.get_data("text8_train_a2f")
w2id,id2w=nlp.build_vocab(Vsize=nVcab)
w2v_dict=nlp.build_w2v(mode="torchnlp",Nvac=nVcab)
nlp.build_pt_emb()
    
dataset=[]
for wrd in nlp.corpus:
    dataset.append(nlp.word_to_id.get(wrd,nlp.word_to_id["UNK"]))
lsize=len(nlp.id_to_word)
print(len(dataset))

nlpb=NLPutil()
nlpb.get_data("text8_valid_gh")
datasetb=[]
for wrd in nlpb.corpus:
    datasetb.append(nlp.word_to_id.get(wrd,nlp.word_to_id["UNK"]))
print(len(datasetb))
nlpc=NLPutil()
nlpc.get_data("text8_test_ij")
datasetc=[]
for wrd in nlpc.corpus:
    datasetc.append(nlp.word_to_id.get(wrd,nlp.word_to_id["UNK"]))
print(len(datasetc))

Initializing NLPUtil
Getting corpus data...
Length of corpus: 10682661
Vocabulary of corpus: 196037
Building vocabulary...
196037 196037
{0: 'the', 1: 'of', 2: 'and', 3: 'one', 4: 'in', 5: 'a', 6: 'to', 7: 'zero', 8: 'nine', 9: 'is', 10: 'two', 11: 'as', 12: 'for', 13: 'eight', 14: 's', 15: 'by', 16: 'that', 17: 'five', 18: 'three', 19: 'was', 20: 'four', 21: 'with', 22: 'six', 23: 'seven', 24: 'on', 25: 'are', 26: 'it', 27: 'from', 28: 'or', 29: 'be', 30: 'an', 31: 'his', 32: 'this', 33: 'which', 34: 'at', 35: 'he', 36: 'not', 37: 'also', 38: 'have', 39: 'has', 40: 'were', 41: 'but', 42: 'other', 43: 'their', 44: 'can', 45: 'they', 46: 'its', 47: 'some', 48: 'first', 49: 'had', 50: 'more', 51: 'all', 52: 'most', 53: 'been', 54: 'such', 55: 'many', 56: 'who', 57: 'used', 58: 'new', 59: 'there', 60: 'when', 61: 'time', 62: 'after', 63: 'into', 64: 'these', 65: 'see', 66: 'only', 67: 'than', 68: 'may', 69: 'american', 70: 'world', 71: 'i', 72: 'would', 73: 'no', 74: 'however', 75: 'about

Except list: length 0
[]
Dimension of embedding: 300
10682661
Initializing NLPUtil
Getting corpus data...
Length of corpus: 3583563
Vocabulary of corpus: 103758
3583563
Initializing NLPUtil
Getting corpus data...
Length of corpus: 2741475
Vocabulary of corpus: 91851
2741475


## POS Tagging with NLTK

In [4]:
# startt = time.time()
# pos_corpus=nltk.pos_tag(nlp.corpus)
# endt = time.time()
# print("Time used in training:", endt - startt)

Time used in training: 362.66596364974976


In [5]:
# startt = time.time()
# pos_corpusb=nltk.pos_tag(nlpb.corpus)
# endt = time.time()
# print("Time used in training:", endt - startt)

Time used in training: 120.75958371162415


In [6]:
# startt = time.time()
# pos_corpusc=nltk.pos_tag(nlpc.corpus)
# endt = time.time()
# print("Time used in training:", endt - startt)

Time used in training: 93.28089237213135


In [4]:
pos_corpus=load_data("pos_corpus_a2f.data")
pos_corpusb=load_data("pos_corpusb_gh.data")
pos_corpusc=load_data("pos_corpus_ij.data")

Data load from  pos_corpus_a2f.data
Data load from  pos_corpusb_gh.data
Data load from  pos_corpus_ij.data


In [8]:
# save_data(pos_corpus,"pos_corpus_a2f.data")
# save_data(pos_corpusb,"pos_corpusb_gh.data")
# save_data(pos_corpusc,"pos_corpus_ij.data")

Data saved to  pos_corpus_a2f.data
Data saved to  pos_corpusb_gh.data
Data saved to  pos_corpus_ij.data


In [5]:
pos_only=[item[1] for item in pos_corpus]
nlp_pos=NLPutil()
nlp_pos.set_corpus(pos_only)
w2id_pos,id2w_pos=nlp_pos.build_vocab(Vsize=nVcab)
pos_only=[w2id_pos[item[1]] for item in pos_corpus]
pos_onlyb=[w2id_pos[item[1]] for item in pos_corpusb]
pos_onlyc=[w2id_pos[item[1]] for item in pos_corpusc]
vocab_out=nlp_pos.nVcab

Initializing NLPUtil
Corpus updated.
Building vocabulary...
{0: 'NN', 1: 'IN', 2: 'JJ', 3: 'DT', 4: 'CD', 5: 'NNS', 6: 'RB', 7: 'CC', 8: 'VBN', 9: 'VBD', 10: 'VBZ', 11: 'VB', 12: 'VBP', 13: 'TO', 14: 'VBG', 15: 'PRP', 16: 'PRP$', 17: 'MD', 18: 'WDT', 19: 'JJR', 20: 'WRB', 21: 'JJS', 22: 'WP', 23: 'RP', 24: 'RBR', 25: 'FW', 26: 'EX', 27: 'RBS', 28: 'NNP', 29: 'PDT', 30: 'WP$', 31: 'NNPS', 32: 'SYM', 33: 'UH', 34: 'POS', 35: '$', 36: '``', 37: "''"}
Collected vocabulary size: 38


In [6]:
cal_entropy(nlp_pos.prior)

2.5537326406914795

In [7]:
cal_entropy_raw(pos_onlyc)

2.5377800637841355

In [8]:
np.log(2**3.6612427)

2.5377800548506815

## buidling POS transforming fact table with WordNet

In [71]:
from nltk.corpus import wordnet as wn
from tqdm import tqdm_notebook

In [42]:
wn.synsets("travel")

[Synset('travel.n.01'),
 Synset('change_of_location.n.01'),
 Synset('locomotion.n.02'),
 Synset('travel.v.01'),
 Synset('travel.v.02'),
 Synset('travel.v.03'),
 Synset('travel.v.04'),
 Synset('travel.v.05'),
 Synset('travel.v.06')]

In [66]:
print(wn.morphy('king', wn.VERB))

None


In [57]:
pos_corpus[0]

('anarchism', 'NN')

In [92]:
v_morphdict=dict([])

In [102]:
cnt_dict=dict([])
cnt_label="VBZ"
for item in tqdm(pos_corpus):
    if item[1]==cnt_label:
        if item[0] in cnt_dict.keys():
            cnt_dict[item[0]]=cnt_dict[item[0]]+1
        else:
            cnt_dict[item[0]]=1

100%|██████████| 10682661/10682661 [00:02<00:00, 3746260.77it/s]


In [103]:
for item in tqdm_notebook(cnt_dict.items()):
    morpgv=wn.morphy(item[0], wn.VERB)
    if morpgv is not None:
        if morpgv in v_morphdict.keys():
            if  v_morphdict[morpgv][cnt_label] is None:
                v_morphdict[morpgv][cnt_label]=item[0]
            else:
                l_item=v_morphdict[morpgv][cnt_label]
                if type(l_item) is list:
                    v_morphdict[morpgv][cnt_label]=l_item.append(item[0])
                else:
                    v_morphdict[morpgv][cnt_label]=[l_item,item[0]]
        else:
            v_morphdict[morpgv]={"VBD":None,"VBG":None,"VBZ":None}
            v_morphdict[morpgv][cnt_label]=item[0]

In [104]:
len(v_morphdict)

5258

In [119]:
for item in v_morphdict.items():
    for sub_item in item[1].items():
        if type(sub_item[1]) is list:
            repl=[]
            for witem in sub_item[1]:
                if witem != item[0]:
                    repl.append(witem)
            if len(repl)==1:
                repl=repl[0]
            v_morphdict[item[0]][sub_item[0]]=repl

In [120]:
for item in v_morphdict.items():
    print(item)

('include', {'VBD': 'included', 'VBG': 'including', 'VBZ': 'includes'})
('differ', {'VBD': 'differed', 'VBG': 'differing', 'VBZ': 'differs'})
('believe', {'VBD': 'believed', 'VBG': 'believing', 'VBZ': 'believes'})
('accord', {'VBD': 'accorded', 'VBG': 'according', 'VBZ': 'accords'})
('be', {'VBD': ['was', 'were'], 'VBG': 'being', 'VBZ': 'bes'})
('concern', {'VBD': 'concerned', 'VBG': 'concerning', 'VBZ': 'concerns'})
('exploit', {'VBD': 'exploited', 'VBG': 'exploiting', 'VBZ': 'exploits'})
('use', {'VBD': 'used', 'VBG': 'using', 'VBZ': 'uses'})
('work', {'VBD': None, 'VBG': 'working', 'VBZ': 'works'})
('say', {'VBD': ['said', 'sayed'], 'VBG': 'saying', 'VBZ': ['says', 'sayings']})
('succeed', {'VBD': 'succeeded', 'VBG': 'succeeding', 'VBZ': 'succeeds'})
('interfere', {'VBD': 'interfered', 'VBG': 'interfering', 'VBZ': 'interferes'})
('rule', {'VBD': 'ruled', 'VBG': 'ruling', 'VBZ': 'rule'})
('associate', {'VBD': 'associated', 'VBG': 'associating', 'VBZ': 'associates'})
('merge', {'VBD':

('publicise', {'VBD': 'publicised', 'VBG': 'publicising', 'VBZ': 'publicises'})
('philander', {'VBD': None, 'VBG': 'philandering', 'VBZ': None})
('badger', {'VBD': 'badgered', 'VBG': 'badgering', 'VBZ': None})
('vet', {'VBD': None, 'VBG': 'vetting', 'VBZ': None})
('delve', {'VBD': 'delved', 'VBG': 'delving', 'VBZ': 'delves'})
('mouth', {'VBD': 'mouthed', 'VBG': 'mouthing', 'VBZ': 'mouth'})
('perplex', {'VBD': 'perplexed', 'VBG': 'perplexing', 'VBZ': None})
('gin', {'VBD': None, 'VBG': 'gin', 'VBZ': None})
('synthesise', {'VBD': 'synthesised', 'VBG': 'synthesising', 'VBZ': 'synthesises'})
('ennoble', {'VBD': 'ennobled', 'VBG': 'ennobling', 'VBZ': 'ennobles'})
('outflank', {'VBD': 'outflanked', 'VBG': 'outflanking', 'VBZ': None})
('naturalise', {'VBD': 'naturalised', 'VBG': 'naturalising', 'VBZ': None})
('misspell', {'VBD': 'misspelled', 'VBG': 'misspelling', 'VBZ': None})
('engulf', {'VBD': 'engulfed', 'VBG': 'engulfing', 'VBZ': None})
('gladden', {'VBD': None, 'VBG': 'gladdening', 'VBZ

In [121]:
save_data(v_morphdict,"v_morphdict.data")

Data saved to  v_morphdict.data


## Bi-GRU Training on POS tag 

In [9]:
lsize_in=300
lsize_out=vocab_out
batch=128
window_size=50
context_dim=5
sample_size=1
cuda_device="cuda:2"
bigru_para={"cuda_device":cuda_device,"self_include_flag":True,"sample_size":sample_size,"mlp_num_layers":2}
# bigru_pos=BiGRU_NLP(lsize_in,30,context_dim,lsize_out,para=bigru_para)
bigru_pos=BiGRU_NLP_VIB(lsize_in,30,context_dim,lsize_out,para=bigru_para)

In [10]:
dataset_sup=dict([])
dataset_sup["dataset"]=dataset
dataset_sup["label"]=pos_only
pt_para={"seqtrain":True,"cuda_flag":True,"loss_clip":20,"cuda_device":cuda_device,"loss_exp_flag":True,
        "pt_emb":nlp.pt_emb,"supervise_mode":True,"reg_lamda":0.2}
ptIf_para={"class":"PyTrain_Interface_sup","version":"vib"}
pt1=PyTrain_Custom(dataset_sup, [lsize_in,lsize_out], bigru_pos, 20000, ptIf_para, learning_rate=1.0e-3, batch=batch, window=window_size, para=pt_para)

Using adam optimizer
cuda:2


In [11]:
pt1.run_training(step=20000,lr=1.0e-3)

Using adam optimizer
Perlexity:  0 47.69749557745132
Perlexity:  400 8.449331485880847
Perlexity:  800 5.699481925121969
Perlexity:  1200 4.756903899727126
Perlexity:  1600 4.339907650960624
Perlexity:  2000 4.06549084530735
Perlexity:  2400 4.058099961393093
Perlexity:  2800 3.870561222076704
Perlexity:  3200 3.702119458584842
Perlexity:  3600 3.6596118225168217
Perlexity:  4000 3.633651777876773
Perlexity:  4400 3.624081924814043
Perlexity:  4800 3.533728898806651
Perlexity:  5200 3.536242989971153
Perlexity:  5600 3.5547052964184256
Perlexity:  6000 3.427887163861814
Perlexity:  6400 3.359593973926157
Perlexity:  6800 3.3239735103257297
Perlexity:  7200 3.488474836436053
Perlexity:  7600 3.4078159292756687
Perlexity:  8000 3.348849096585669
Perlexity:  8400 3.419035705587244
Perlexity:  8800 3.3687087621627314
Perlexity:  9200 3.26006287776022
Perlexity:  9600 3.3658089870621555
Perlexity:  10000 3.373330517143128
Perlexity:  10400 3.3104963603238153
Perlexity:  10800 3.233038271947

In [12]:
pt1.do_eval(step_eval=600)

Evaluation Perplexity:  1.6287524299815648
Time used in evaluation: 7.572697639465332


1.6287524299815648

In [22]:
# pt1.do_test(step_test=600)

In [13]:
#### Test Only
datasetc_sup=dict([])
datasetc_sup["dataset"]=datasetc
datasetc_sup["label"]=pos_onlyc
ptT=PyTrain_Custom(datasetc_sup, [lsize_in,lsize_out], bigru_pos, 20000, ptIf_para, learning_rate=1.0e-3, batch=64, window=window_size, para=pt_para)
perp=ptT.do_eval(step_eval=600,eval_mem_flag=True)

Using adam optimizer
cuda:2
Evaluation Perplexity:  1.6597282218131164
Time used in evaluation: 6.567547798156738


In [14]:
np.log(perp)

0.5066538671652476

In [ ]:
# ptT.do_test(step_test=600)

In [30]:
# save_model(bigru_pos,"bigru_pos_vib_40K_r0p1.model")

Model saved to  bigru_pos_vib_40K_r0p1.model


## Output Entropy Consistency Check

In [15]:
cdim=context_dim
mat_theta=np.array(ptT.evalmem[3]).reshape(-1,cdim)
mat_mu=np.array(ptT.evalmem[4]).reshape(-1,cdim)

length=len(mat_theta)
ent=0
for ii in tqdm_notebook(range(length)):
    ent=ent+cal_entropy_gauss(list(mat_theta[ii]))
glent=ent/length
print("Gauss layer entropy:",ent/length)


Gauss layer entropy: 3.701062360317586


In [16]:
cont_mat=np.array(ptT.evalmem[2]).reshape(-1,cdim)
print("Data number:",len(cont_mat))

tot_ent=0
for ii in range(cdim):
    ent=cal_entropy_continous_raw(cont_mat[:,ii],bins=100)
    tot_ent=tot_ent+ent
print("Context entropy",tot_ent)
entdiff=tot_ent-glent
print("Extropy Diff",entdiff)

Data number: 1920000
Context entropy 6.9422791499644445
Extropy Diff 3.2412167896468587


In [17]:
out_mat=np.array(ptT.evalmem[5])
out_mat=out_mat.reshape(-1,sample_size,vocab_out)
out_mat=np.exp(out_mat)
out_mat=np.mean(out_mat,axis=1)
resmat=cal_entropy(out_mat)
outent=np.mean(resmat)
print("Calculated output entropy:",outent)

prior_ent=cal_entropy(nlp_pos.prior)
print("Prior ent:",prior_ent)

Calculated output entropy: 0.5100340368675645
Prior ent: 2.5537326406914795


In [18]:
# Checking output prior v.s. Data prior
resl=[]
for ii in tqdm_notebook(range(len(out_mat))):
    res=sample_id(out_mat[ii],(10,)).numpy()
    resl.append(res)
resl=np.array(resl)
print(resl.shape)
calpent=cal_entropy_raw(resl.reshape(-1))
print("Calculated prior:",calpent)

# print("Perp mutual information:",prior_ent-np.log(perp))

print("Output mutual information:",calpent-outent)

print(entdiff-(calpent-outent))
print("IB theory:",entdiff-(calpent-outent)>0)


(1920000, 10)
Calculated prior: 2.541950881215846
Output mutual information: 2.0319168443482813
1.2092999452985773
IB theory: True


In [19]:
prec=(calpent-outent)/entdiff
recl=(calpent-outent)/prior_ent
F1=2*prec*recl/(prec+recl)
print("Precision:",prec)
print("Recall:",recl)
print("Information F1:",F1)

Precision: 0.6268993949552092
Recall: 0.7956654553305529
Information F1: 0.7012716396490272


In [20]:
#Precision: 0.6523802113890129
#Recall: 0.8039280996394168
#Information F1: 0.7202688875873506

## Output Entropy Consistency Check /old

In [90]:
out_mat=np.array(ptT.evalmem[5])
out_mat.shape

(600, 50, 64, 64, 38)

In [91]:
out_mat=out_mat.reshape(-1,64,38)
out_mat.shape

(1920000, 64, 38)

In [92]:
out_mat=np.exp(out_mat)

In [94]:
out_mat=np.mean(out_mat,axis=1)
out_mat.shape

(1920000, 38)

In [112]:
out_mat[2,:]

array([7.2856802e-01, 3.6519045e-20, 2.0649230e-01, 3.4473767e-03,
       1.2914486e-09, 1.3485234e-02, 1.8060255e-05, 7.9917735e-14,
       2.4031948e-04, 2.0732025e-04, 9.1468362e-04, 1.0350704e-02,
       1.7043749e-02, 1.2250032e-07, 1.7424297e-02, 6.0400475e-07,
       3.9478502e-07, 3.2289077e-07, 2.9941180e-08, 2.5642541e-04,
       1.5027501e-07, 4.3349851e-06, 1.7426717e-07, 1.2168194e-08,
       6.9480855e-05, 6.9192791e-04, 1.5699284e-07, 4.7204392e-07,
       5.7125004e-04, 2.0928240e-04, 5.6908004e-08, 1.9971287e-06,
       5.3218520e-07, 1.5733832e-08, 1.5691245e-07, 2.5749243e-08,
       6.5172139e-09, 2.9419156e-09], dtype=float32)

In [95]:
resmat=cal_entropy(out_mat)
len(resmat)

1920000

In [96]:
np.mean(resmat)

1.2147264676658938

In [88]:
1.7371846644613749-0.606650134073827

1.130534530387548

In [97]:
2.5377800548506815-1.2147264676658938

1.3230535871847877

In [56]:
1.4669429718239266-0.35886600072844194

1.1080769710954848

In [57]:
2.5377800548506815-1.2436261432050135

1.294153911645668

In [31]:
1.4677734397939906-0.35925732210757816

1.1085161176864124

In [32]:
2.5377800548506815-1.243095212757116

1.2946848420935655

In [113]:
2.5377800548506815-1.3275332135632956

1.210246841287386

In [114]:
1.5062682325181005-0.37568676780703736

1.1305814647110632

In [113]:
# Checking output prior v.s. Data prior
resl=[]
for ii in tqdm_notebook(range(len(out_mat))):
    res=sample_id(out_mat[ii],(10,)).numpy()
    resl.append(res)
resl=np.array(resl)
print(resl.shape)
cal_entropy_raw(resl.reshape(-1))


(1920000, 10)


2.560995596797616

## Entropy Check

In [ ]:
## Result dim=5
## Hk=2.5377800548506 res=0.2308968534
## beta=0.2 H(i|k)=5.40634046787 H(i)=6.88502266 I=1.478682  res=0.48964996
## beta=0.15 H(i|k)=5.2461315  H(i)=6.8827212 I=1.6365897 res=0.4378229
## beta=0.1 H(i|k)=5.041667783 H(i)=6.9158477 I=1.8741799 res=0.38867505
## beta=0.08 H(i|k)=4.8861000 H(i)=6.8546164 I=1.9685164 res=0.362102
## beta=0.05 H(i|k)=4.58959918 H(i)=6.782526549 I=2.192927369 res = 0.32097623
## beta=0.02 H(i|k)=3.8957862 H(i)=6.81511528 I=2.919329 res=0.28524466

## Result dim=10
## beta=0.2 H(i|k)=12.553735 H(i)=13.95712059 I=1.40338559 res=0.50113135
## beta=0.15 H(i|k)=12.336273 H(i)=13.8866799 I=1.5504069 res=0.441079684
## beta=0.1 H(i|k)=12.1648569 H(i)=13.9988189 I=1.833962 res=0.3917166
## beta=0.08 H(i|k)=11.90123007 H(i)=13.8522579 I=1.95102783 res=0.3532165997
## beta=0.05 H(i|k)=11.56218147 H(i)=13.8891091 I=2.32692763 res=0.31532219
## beta=0.02 H(i|k)=10.7989604 H(i)=13.96900735 I=3.17004695 res=0.274927158

## Result dim=20
## beta=0.2 H(i|k)=26.7656519 H(i)=28.2012864 I=1.4356345 res=0.49832557
## beta=0.15 H(i|k)=26.5892173 H(i)=28.16647022 I=1.57725292 res=0.4456432922
## beta=0.1 H(i|k)=26.2716403 H(i)=28.0996679 I=1.8280275 res=0.3838920998
## beta=0.08 H(i|k)=26.14235938 H(i)=28.12058865 I=1.97822927 res=0.365701515
## beta=0.05 H(i|k)=25.832600496 H(i)=28.15494155 I=2.322341054 res=0.3269281326
## beta=0.02 H(i|k)=24.8897738 H(i)=27.82383213 T= 2.93405832 res=0.270175362

In [79]:
cdim=1

In [80]:
mat_theta=np.array(ptT.evalmem[3]).reshape(-1,cdim)

In [81]:
mat_mu=np.array(ptT.evalmem[4]).reshape(-1,cdim)

In [412]:
# cal_entropy_gauss([1,1,1,1,1])

In [82]:
len(mat_theta)

1920000

In [83]:
mat_theta.shape

(1920000, 1)

In [101]:
# length=100000
length=len(mat_theta)
ent=0
for ii in tqdm_notebook(range(length)):
    ent=ent+cal_entropy_gauss(list(mat_theta[ii]))
print(ent/length)


0.606650134073827


In [59]:
# splen=100
# datas=[]
# for iil in tqdm_notebook(range(length)):
#     mu=mat_mu[iil]
#     theta=mat_theta[iil]
#     data=np.random.normal(loc=mu,scale=theta,size=splen)
#     datas.append(data)
# datas=np.array(datas).reshape(-1)
# print(len(datas))
# cal_entropy_continous_raw(datas,bins=10000)

In [102]:
np.array(ptT.evalmem[2]).shape

(600, 50, 64, 64, 1)

In [103]:
cont_mat=np.array(ptT.evalmem[2]).reshape(-1,cdim)

In [104]:
cont_mat.shape

(122880000, 1)

In [95]:
# plt.hist(mat_mu[:,0],bins=1000)
# plt.show()

In [109]:
tot_ent=0
for ii in range(cdim):
    ent=cal_entropy_continous_raw(cont_mat[:,ii],bins=10000)
    tot_ent=tot_ent+ent
print(tot_ent)

1.7371846644613749


In [432]:
## beta=0.2 H(i|k)=5.40634046787 H(i)=6.88502266 I=1.478682  res=0.48964996
## beta=0.15 H(i|k)=5.2461315  H(i)=6.8827212 I=1.6365897 res=0.4378229
## beta=0.1 H(i|k)=5.041667783 H(i)=6.9158477 I=1.8741799 res=0.38867505
## beta=0.08 H(i|k)=4.8861000 H(i)=6.8546164 I=1.9685164 res=0.362102
## beta=0.05 H(i|k)=4.58959918 H(i)=6.782526549 I=2.192927369 res = 0.32097623
## beta=0.02 H(i|k)=3.8957862 H(i)=6.81511528 I=2.919329 res=0.28524466

vec5=[[0.02,2.919329,0.28524466],
     [0.05,2.192927369,0.32097623],
     [0.08,1.9685164,0.362102],
     [0.1,1.8741799,0.38867505],
     [0.15,1.6365897,0.4378229],
     [0.2,1.478682,0.48964996]]

## Result dim=10
## beta=0.2 H(i|k)=12.553735 H(i)=13.95712059 I=1.40338559 res=0.50113135
## beta=0.15 H(i|k)=12.336273 H(i)=13.8866799 I=1.5504069 res=0.441079684
## beta=0.1 H(i|k)=12.1648569 H(i)=13.9988189 I=1.833962 res=0.3917166
## beta=0.08 H(i|k)=11.90123007 H(i)=13.8522579 I=1.95102783 res=0.3532165997
## beta=0.05 H(i|k)=11.56218147 H(i)=13.8891091 I=2.32692763 res=0.31532219
## beta=0.02 H(i|k)=10.7989604 H(i)=13.96900735 I=3.17004695 res=0.274927158

vec10=[[0.02,3.17004695,0.274927158],
     [0.05,2.32692763,0.31532219],
     [0.08,1.95102783,0.3532165997],
     [0.1,1.833962,0.3917166],
     [0.15,1.5504069,0.441079684],
     [0.2,1.40338559,0.50113135]]

## Result dim=20
## beta=0.2 H(i|k)=26.7656519 H(i)=28.2012864 I=1.4356345 res=0.49832557
## beta=0.15 H(i|k)=26.5892173 H(i)=28.16647022 I=1.57725292 res=0.4456432922
## beta=0.1 H(i|k)=26.2716403 H(i)=28.0996679 I=1.8280275 res=0.3838920998
## beta=0.08 H(i|k)=26.14235938 H(i)=28.12058865 I=1.97822927 res=0.365701515
## beta=0.05 H(i|k)=25.832600496 H(i)=28.15494155 I=2.322341054 res=0.3269281326
## beta=0.02 H(i|k)=24.8897738 H(i)=27.82383213 T= 2.93405832 res=0.270175362

vec20=[[0.02,2.93405832,0.270175362],
     [0.05,2.322341054,0.3269281326],
     [0.08,1.97822927,0.365701515],
     [0.1,1.8280275,0.3838920998],
     [0.15,1.57725292,0.4456432922],
     [0.2,1.4356345,0.49832557]]

In [433]:
vec5=np.array(vec5)
vec10=np.array(vec10)
vec20=np.array(vec20)

In [459]:
fig, ax1 = plt.subplots()

ax2 = ax1.twinx()
ax1.plot(vec5[:,0], vec5[:,1], 'g-^')
ax2.plot(vec5[:,0], vec5[:,2], 'b-^')
ax1.plot(vec10[:,0], vec10[:,1], 'g--o')
ax2.plot(vec10[:,0], vec10[:,2], 'b--o')
ax1.plot(vec20[:,0], vec20[:,1], 'g-.*')
ax2.plot(vec20[:,0], vec20[:,2], 'b-.*')

# ax1.plot(vec5[:,0], vec5[:,1]+vec5[:,2], 'r-^')
# ax1.plot(vec10[:,0], vec10[:,1]+vec10[:,2], 'r--o')
# ax1.plot(vec20[:,0], vec20[:,1]+vec20[:,2], 'r-.*')

ax1.set_xlabel('beta')
ax1.set_ylabel('Mutual Information', color='g')
ax2.set_ylabel('Cross Entropy', color='b')

ax1.legend(["Dim5","Dim10","Dim20"],loc=5)
# ax2.legend(["Dim5","Dim10","Dim20"],loc=6)
plt.show()

In [98]:
cal_entropy_gauss([1])

1.4189385332046727

In [119]:
nnn=np.random.normal(loc=0,scale=1,size=10000)
cal_entropy_continous_raw(nnn,bins=100)

1.4194588065484364

## 2-bin gaussian mixture

In [61]:
np.random.normal(loc=1,scale=1,size=1)

array([0.92262678])

In [129]:
gmsamples=[]
sids=[]
for ii in range(1000000):
    sid=np.random.rand()
    sids.append(sid)
    if sid>0.5:
        gmsamples.append(np.random.normal(loc=2,scale=1,size=1))
    else:
        gmsamples.append(np.random.normal(loc=-2,scale=1,size=1))

In [130]:
npgs=np.array(gmsamples).reshape(-1)

In [137]:
plt.hist(npgs,bins=1000)
plt.show()

In [134]:
ent=cal_entropy_continous_raw(npgs,bins=10000)
print(ent)

2.047326730921048


In [135]:
cal_entropy_gauss(1.0)

1.4189385332046727

In [136]:
(2.04732-1.41893)/np.log(2)

0.9065751367442157

## Getting POS Codebook

In [371]:
### Result check
cntm=np.transpose(cntm,(0,2,1,3)).reshape((-1,context_dim))
print(cntm.shape)
lb=np.array(ptT.evalmem[1]).reshape(-1)
print(lb.shape)
lbs=[]
for ii in range(len(lb)):
    lbs.append(id2w_pos[lb[ii]])
print(len(lbs))

(3840000, 5)
(3840000,)
3840000


In [372]:
res=[]
for item in cntm:
    res.append(tuple(item))

In [373]:
len(set(res))

966

In [374]:
import collections
counter = collections.Counter(res)

In [375]:
count_pairs = sorted(counter.items(), key=lambda x: (-x[1], x[0]))
words, counts = list(zip(*count_pairs))

In [376]:
cnt=np.array(counts)
cnt=cnt/np.max(cnt)
cnt

array([1.00000000e+00, 6.82168412e-01, 5.48603584e-01, 4.39320002e-01,
       3.15088583e-01, 2.74238867e-01, 2.39252711e-01, 2.34321466e-01,
       1.90060973e-01, 1.80938169e-01, 1.77761112e-01, 1.77363017e-01,
       1.72326469e-01, 1.72154389e-01, 1.55108205e-01, 1.50790797e-01,
       1.45890372e-01, 1.31150572e-01, 1.31104342e-01, 1.20836052e-01,
       9.87379095e-02, 9.25841265e-02, 8.55519656e-02, 8.35306687e-02,
       8.34176610e-02, 8.32969483e-02, 7.61106859e-02, 7.55738993e-02,
       7.55738993e-02, 7.27975056e-02, 7.19294010e-02, 7.06272441e-02,
       6.87215233e-02, 6.73525892e-02, 6.42602876e-02, 6.29915193e-02,
       6.21336881e-02, 6.12835620e-02, 5.83838872e-02, 5.83453618e-02,
       5.80731160e-02, 5.73128824e-02, 5.65860374e-02, 5.58720342e-02,
       5.58566240e-02, 5.52659020e-02, 5.51991247e-02, 5.05093052e-02,
       4.99211514e-02, 4.77380482e-02, 4.25910611e-02, 4.25833560e-02,
       4.16638843e-02, 4.15611500e-02, 4.14686892e-02, 3.94756443e-02,
      

In [377]:
cal_entropy(cnt)

6.3905086451930195

In [402]:
tpkcnt=dict([])
for ii in tqdm_notebook(range(len(cntm))):
    tpk=tuple(cntm[ii])
    if tpk not in tpkcnt:
        initdict=dict([])
        for ii in range(len(id2w_pos)):
            initdict[id2w_pos[ii]]=0
        tpkcnt[tpk]=initdict
    else:
        tpkcnt[tpk][lbs[ii]]=tpkcnt[tpk][lbs[ii]]+1

In [379]:
words

((0.3, 0.0, 0.0, 0.0, 0.0),
 (0.3, 0.3, 0.0, 0.0, 0.0),
 (0.0, 0.3, 0.0, 0.3, 0.0),
 (0.3, 0.0, 0.0, 0.2, 0.1),
 (0.3, 0.0, 0.0, 0.3, 0.1),
 (0.0, 0.3, 0.0, 0.2, 0.0),
 (0.2, 0.0, 0.0, 0.2, 0.2),
 (0.2, 0.0, 0.0, 0.2, 0.1),
 (0.0, 0.0, 0.0, 0.0, 0.0),
 (0.2, 0.3, 0.0, 0.0, 0.0),
 (0.3, 0.0, 0.0, 0.2, 0.2),
 (0.2, 0.1, 0.0, 0.2, 0.2),
 (0.2, 0.0, 0.0, 0.0, 0.3),
 (0.0, 0.0, 0.0, 0.2, 0.0),
 (0.0, 0.2, 0.0, 0.3, 0.0),
 (0.3, 0.0, 0.0, 0.0, 0.3),
 (0.0, 0.0, 0.0, 0.0, 0.6),
 (0.2, 0.1, 0.0, 0.0, 0.3),
 (0.2, 0.0, 0.0, 0.1, 0.2),
 (0.2, 0.1, 0.0, 0.1, 0.2),
 (0.0, 0.0, 0.0, 0.0, 0.3),
 (0.0, 0.2, 0.0, 0.2, 0.0),
 (0.2, 0.1, 0.0, 0.2, 0.1),
 (0.0, 0.3, 0.0, 0.0, 0.2),
 (0.0, 0.2, 0.0, 0.0, 0.2),
 (0.0, 0.5, 0.0, 0.0, 0.5),
 (0.0, 0.1, 0.0, 0.0, 0.6),
 (0.1, 0.0, 0.0, 0.4, 0.4),
 (0.3, 0.1, 0.0, 0.0, 0.3),
 (0.2, 0.0, 0.0, 0.3, 0.1),
 (0.0, 0.0, 0.0, 0.5, 0.2),
 (0.0, 0.0, 0.0, 0.2, 0.2),
 (0.0, 0.0, 0.0, 0.6, 0.2),
 (0.1, 0.0, 0.0, 0.2, 0.2),
 (0.0, 0.0, 0.0, 0.1, 0.0),
 (0.0, 0.5, 0.0, 0.0

In [380]:
counts

(389354,
 265605,
 213601,
 171051,
 122681,
 106776,
 93154,
 91234,
 74001,
 70449,
 69212,
 69057,
 67096,
 67029,
 60392,
 58711,
 56803,
 51064,
 51046,
 47048,
 38444,
 36048,
 33310,
 32523,
 32479,
 32432,
 29634,
 29425,
 29425,
 28344,
 28006,
 27499,
 26757,
 26224,
 25020,
 24526,
 24192,
 23861,
 22732,
 22717,
 22611,
 22315,
 22032,
 21754,
 21748,
 21518,
 21492,
 19666,
 19437,
 18587,
 16583,
 16580,
 16222,
 16182,
 16146,
 15370,
 15007,
 14650,
 14288,
 14242,
 14191,
 14085,
 13949,
 12534,
 11777,
 11619,
 10914,
 10580,
 10560,
 10540,
 10385,
 9737,
 9545,
 8906,
 8845,
 8770,
 8679,
 8651,
 8591,
 7695,
 7680,
 7539,
 6487,
 6376,
 6357,
 6252,
 6232,
 5992,
 5937,
 5930,
 5922,
 5695,
 5524,
 5119,
 5008,
 4972,
 4919,
 4916,
 4767,
 4731,
 4575,
 4540,
 4482,
 4395,
 4352,
 4347,
 4229,
 4062,
 3895,
 3792,
 3769,
 3647,
 3563,
 3514,
 3486,
 3451,
 3447,
 3395,
 3239,
 3235,
 3177,
 3167,
 3162,
 3157,
 3113,
 3108,
 3090,
 3078,
 3032,
 2995,
 2986,
 2984,

In [381]:
len(tpkcnt)

966

In [382]:
nlp_pos.prior/nlp_pos.prior[0]

array([1.00000000e+00, 5.11620156e-01, 4.57662637e-01, 4.15691573e-01,
       3.83157416e-01, 2.90322720e-01, 1.39705986e-01, 1.32473160e-01,
       1.18422149e-01, 1.18240179e-01, 1.00552907e-01, 8.97132966e-02,
       8.68949195e-02, 7.89774451e-02, 6.37552213e-02, 5.46142624e-02,
       3.42501018e-02, 2.58425262e-02, 2.23131680e-02, 1.09976057e-02,
       1.00512032e-02, 1.00405915e-02, 8.64260415e-03, 6.49747757e-03,
       6.05178802e-03, 5.98025760e-03, 5.16905547e-03, 4.03203620e-03,
       3.64962356e-03, 2.16124057e-03, 7.25129816e-04, 1.64677178e-04,
       8.52862710e-05, 3.77303319e-05, 2.55465789e-05, 1.53279473e-05,
       5.89536436e-06, 2.35814574e-06])

In [383]:
for idid in range(len(words)):
    div=tpkcnt[words[idid]]["NN"]
    if div==0:
        div=1
    for (k,v) in tpkcnt[words[idid]].items():
        tpkcnt[words[idid]][k]=tpkcnt[words[idid]][k]

In [388]:
idid=3
print(words[idid])
tpkcnt[words[idid]]

(0.3, 0.0, 0.0, 0.2, 0.1)


{'NN': 1.0,
 'IN': 6.50375739802404e-05,
 'JJ': 0.0044343800441073,
 'DT': 1.18250134509528e-05,
 'CD': 0.004889643061968983,
 'NNS': 0.0006444632330769276,
 'RB': 7.09500807057168e-05,
 'CC': 1.77375201764292e-05,
 'VBN': 5.32125605292876e-05,
 'VBD': 3.54750403528584e-05,
 'VBZ': 4.73000538038112e-05,
 'VB': 0.0001064251210585752,
 'VBP': 0.0004020504573323952,
 'TO': 0.0,
 'VBG': 0.00041387547078334803,
 'PRP': 5.9125067254764e-06,
 'PRP$': 0.0,
 'MD': 1.18250134509528e-05,
 'WDT': 1.18250134509528e-05,
 'JJR': 4.1387547078334805e-05,
 'WRB': 0.0,
 'JJS': 0.0,
 'WP': 0.0,
 'RP': 0.0,
 'RBR': 2.36500269019056e-05,
 'FW': 2.9562533627382e-05,
 'EX': 0.0,
 'RBS': 0.0,
 'NNP': 1.77375201764292e-05,
 'PDT': 0.0,
 'WP$': 0.0,
 'NNPS': 0.0,
 'SYM': 0.0,
 'UH': 0.0,
 'POS': 0.0,
 '$': 0.0,
 '``': 0.0,
 "''": 0.0}

In [392]:
pltsne(cntm[:1000,:]+0.01*np.random.random((1000,5)),D=2,perp=10,labels=lbs[:1000])

TSNE(angle=0.5, early_exaggeration=12.0, init='pca', learning_rate=200.0,
     method='exact', metric='euclidean', min_grad_norm=1e-07, n_components=2,
     n_iter=5000, n_iter_without_progress=300, perplexity=10, random_state=None,
     verbose=0)

In [394]:
len(tpkcnt)

966

In [411]:
cwl=len(words)
posl=len(id2w_pos)
reprmat=np.zeros((len(words),len(id2w_pos)))
for ii in range(cwl):
    for jj in range(posl):
        reprmat[ii,jj]=tpkcnt[words[ii]][id2w_pos[jj]]

In [482]:
trg="NN"
tgid=w2id_pos[trg]
sorttp=zip(np.linspace(0,cwl-1,cwl),reprmat[:,tgid])
count_pairs = sorted(sorttp, key=lambda x: -x[1])

In [472]:
reprmat=reprmat/np.sum(reprmat,axis=0)

array([[1.12254666e-03, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [6.48246385e-06, 4.72061482e-01, 4.79550757e-06, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [480]:
cdbook=dict([])
for iic in range(len(id2w_pos)):
    cdbook[id2w_pos[iic]]=np.sum(np.array(words)*reprmat[:,iic].reshape(-1,1),axis=0)

In [488]:
# save_data(cdbook,"cdbook_1.data")

Data saved to  cdbook_1.data


In [481]:
cdbook

{'NN': array([0.25261298, 0.0225741 , 0.        , 0.21789755, 0.14079923]),
 'IN': array([0.00458355, 0.2724262 , 0.        , 0.26383709, 0.00562882]),
 'JJ': array([0.22168913, 0.04553742, 0.        , 0.03866114, 0.25386147]),
 'DT': array([0.27691451, 0.29303393, 0.        , 0.00109843, 0.00049936]),
 'CD': array([2.98133625e-01, 7.81719942e-05, 0.00000000e+00, 4.48453979e-03,
        6.08329101e-04]),
 'NNS': array([0.10986436, 0.04857934, 0.        , 0.3306559 , 0.39032489]),
 'RB': array([0.0212643 , 0.23550252, 0.        , 0.01898678, 0.17064613]),
 'CC': array([0.00219375, 0.04205759, 0.        , 0.00191322, 0.56856737]),
 'VBN': array([0.02438429, 0.00452785, 0.        , 0.01642115, 0.01744261]),
 'VBD': array([0.02024911, 0.00471298, 0.        , 0.1721125 , 0.02215512]),
 'VBZ': array([0.01484927, 0.00764513, 0.        , 0.48974793, 0.20479242]),
 'VB': array([0.01820732, 0.004809  , 0.        , 0.02950219, 0.24843971]),
 'VBP': array([0.0521721 , 0.02013154, 0.        , 0.182

In [158]:
cal_entropy_raw_ND_discrete(cntm)

6.391629341448024

In [159]:
np.exp(np.log(63)-(np.log(3.6)-np.log(1.4)))

24.500000000000004

In [160]:
np.exp(np.log(77)-(np.log(3.6)-np.log(1.4)))

29.94444444444446

## Bi-GRU Training on POS tag Adversary model

In [102]:
lsize_in=300
lsize_out=vocab_out
advlsize_out=nVcab+1
batch=64
window_size=50
cuda_device="cuda:0"
bigru_para={"cuda_device":cuda_device,"self_include_flag":True,"adv_mode_flag":True,"adv_output_size":advlsize_out}
bigru_pos=BiGRU_NLP(lsize_in,30,15,lsize_out,batch,para=bigru_para)

In [103]:
dataset_sup=dict([])
dataset_sup["dataset"]=dataset
dataset_sup["label"]=pos_only
dataset_sup["advlabel"]=dataset
pt_para={"seqtrain":True,"cuda_flag":True,"loss_clip":20,"cuda_device":cuda_device,"loss_exp_flag":True,
        "pt_emb":nlp.pt_emb,"supervise_mode":True,"reg_lamda":0.01}
ptIf_para={"class":"PyTrain_Interface_advsup"}
pt1=PyTrain_Custom(dataset_sup, [lsize_in,lsize_out], bigru_pos, 20000, ptIf_para, learning_rate=1.0e-3, batch=batch, window=window_size, para=pt_para)

Using adam optimizer
cuda:0


In [104]:
pt1.run_training(step=10000,lr=1.0e-3)

Using adam optimizer
Perlexity:  0 36.999405042428144
Perlexity:  200 3.0880652456254554
Perlexity:  400 1.804648914208581
Perlexity:  600 1.4313487571178316
Perlexity:  800 1.251744871649741
Perlexity:  1000 1.0604146390608293
Perlexity:  1200 0.9540154993064025
Perlexity:  1400 0.8475172078669325
Perlexity:  1600 0.7706063632685852
Perlexity:  1800 0.7211969667510558
Perlexity:  2000 0.6809399482443189
Perlexity:  2200 0.6538123119812055
Perlexity:  2400 0.6180644443576828
Perlexity:  2600 0.5699284711245208
Perlexity:  2800 0.5513142630264771
Perlexity:  3000 0.529714938348822
Perlexity:  3200 0.4968386807807863
Perlexity:  3400 0.449615216569868
Perlexity:  3600 0.43207950842161524
Perlexity:  3800 0.40695875374748053
Perlexity:  4000 0.3947347376968502
Perlexity:  4200 0.37970793501093797
Perlexity:  4400 0.3531542315401726
Perlexity:  4600 0.3353764521488983
Perlexity:  4800 0.3105229429720461
Perlexity:  5000 0.30182722327755973
Perlexity:  5200 0.2768206535118402
Perlexity:  54

In [105]:
pt1.do_eval(step_eval=600)

Evaluation Perplexity:  1.3308531024121055
Time used in evaluation: 7.6443986892700195


1.3308531024121055

In [107]:
# pt1.do_test(step_test=600)

In [108]:
#### Test Only
datasetb_sup=dict([])
datasetb_sup["dataset"]=datasetb
datasetb_sup["label"]=pos_onlyb
datasetb_sup["advlabel"]=datasetb
ptT=PyTrain_Custom(datasetb_sup, [lsize_in,lsize_out], bigru_pos, 20000, ptIf_para, learning_rate=1.0e-3, batch=64, window=window_size, para=pt_para)
ptT.do_eval(step_eval=600)

Using adam optimizer
cuda:0
Evaluation Perplexity:  1.342137545910216
Time used in evaluation: 7.386909008026123


1.342137545910216

In [ ]:
ptT.do_test(step_test=600)

In [111]:
#### Test Only
datasetc_sup=dict([])
datasetc_sup["dataset"]=datasetc
datasetc_sup["label"]=pos_onlyc
datasetc_sup["advlabel"]=datasetc
ptT=PyTrain_Custom(datasetc_sup, [lsize_in,lsize_out], bigru_pos, 20000, ptIf_para, learning_rate=1.0e-3, batch=64, window=window_size, para=pt_para)
ptT.do_eval(step_eval=600)

Using adam optimizer
cuda:0
Evaluation Perplexity:  1.3512125893718039
Time used in evaluation: 7.3829545974731445


1.3512125893718039

In [ ]:
ptT.do_test(step_test=600)

## Cooperative Bi-GRU Languag Model

In [213]:
# save_model(bigru_pos,"bigru_pos_1.model")
# save_model(bigru_coop,"bigru_coop_1.model")

Model saved to  bigru_pos_1.model
Model saved to  bigru_coop_1.model


In [25]:
cal_entropy(nlp.prior)

10.197888913442997

In [544]:
# bigru_coop=load_model(bigru_coop,"bigru_coop_1.model")

Model load from  bigru_coop_1.model


In [35]:
lsize_in=300
lsize_out=nVcab+1
batch=64
window_size=50
context_dim=30
bigru_para_coop={"cuda_device":cuda_device,"self_include_flag":False,"gate_mode_flag":True}
bigru_train=BiGRU_NLP(lsize_in,100,context_dim,lsize_out,para=bigru_para_coop)

In [36]:
bigru_coop=ABS_NLP_COOP(bigru_train,bigru_pos)

In [47]:
pt_para={"loss_clip":1000,"cuda_device":cuda_device,"pt_emb":nlp.pt_emb,"reg_lamda":20.0}
ptIf_para={"class":"PyTrain_Interface_continous","version":1}
pt1=PyTrain_Custom(dataset, [lsize_in,lsize_out], bigru_coop, 20000, ptIf_para, learning_rate=1.0e-3, batch=batch, window=window_size, para=pt_para)

Using adam optimizer
cuda:2


In [60]:
pt1.run_training(step=20000,lr=1.0e-3)

Using adam optimizer
Perlexity:  0 187.76825407530507
Perlexity:  400 154.73252466827955
Perlexity:  800 164.3833146302822
Perlexity:  1200 171.9245342058837
Perlexity:  1600 172.01900111689648
Perlexity:  2000 157.16635952494744
Perlexity:  2400 190.45925474323647
Perlexity:  2800 163.19501677432518
Perlexity:  3200 185.64060625215924
Perlexity:  3600 176.95056832133355
Perlexity:  4000 206.60891628958643
Perlexity:  4400 184.7763642002498
Perlexity:  4800 155.33242728513468
Perlexity:  5200 167.09710259951953
Perlexity:  5600 174.58141348453822
Perlexity:  6000 176.73360044011451
Perlexity:  6400 183.57689201338147
Perlexity:  6800 185.4663915016549
Perlexity:  7200 195.56451995842107
Perlexity:  7600 168.96097294607875
Perlexity:  8000 186.62071457696288
Perlexity:  8400 184.2753563558185
Perlexity:  8800 182.41051563352343
Perlexity:  9200 173.92848332853816
Perlexity:  9600 174.89511959492984
Perlexity:  10000 173.58464106159832
Perlexity:  10400 183.93798370236584
Perlexity:  108

In [61]:
pt1.do_test()

Correct rate:  0.3610422926592984
Time used in test: 6.35131573677063


0.3610422926592984

In [75]:
pt1._postscript()

In [63]:
pt1.do_eval(step_eval=600)

Evaluation Perplexity:  47.65322352975383
Time used in evaluation: 21.159218311309814


47.65322352975383

In [64]:
# save_model(bigru_coop,"bigru_coop.temp")

In [65]:
#### Test Only
ptT=PyTrain_Custom(datasetb, [lsize_in,lsize_out], bigru_coop, 20000, ptIf_para, learning_rate=1.0e-3, batch=64, window=window_size, para=pt_para)
ptT.do_eval(step_eval=600)

Using adam optimizer
cuda:2
Evaluation Perplexity:  43.96320275487483
Time used in evaluation: 21.00189185142517


43.96320275487483

In [66]:
ptT.do_test(step_test=1200)

Correct rate:  0.3732350241739735
Time used in test: 25.392481803894043


0.3732350241739735

In [71]:
#### Test Only
ptT=PyTrain_Custom(datasetc, [lsize_in,lsize_out], bigru_coop, 20000, ptIf_para, learning_rate=1.0e-3, batch=64, window=window_size, para=pt_para)
ptT.do_eval(step_eval=1200)

Using adam optimizer
cuda:2
Evaluation Perplexity:  45.55997980838988
Time used in evaluation: 43.40083980560303


45.55997980838988

In [72]:
ptT.do_test(step_test=1200)

Correct rate:  0.36926513272848677
Time used in test: 25.444421529769897


0.36926513272848677

In [73]:
### Result check
cntm=np.array(ptT.evalmem[2]).reshape((-1,context_dim))
print(cntm.shape)

(3840000, 30)


In [74]:
cal_entropy_raw_ND_discrete(cntm)

16.8672636427109

In [76]:
# save_model(bigru_coop,"bigru_coop_engreg_1_selfnotInclude_60K.model")

Model saved to  bigru_coop_engreg_1_selfnotInclude_60K.model


## POS information check

In [80]:
bigru_train=bigru_coop.trainer
bigru_coop_pos=ABS_NLP_COOP(None,bigru_train,output_size=vocab_out)
pt_para={"seqtrain":True,"cuda_flag":True,"loss_clip":200,"cuda_device":cuda_device,"loss_exp_flag":True,
        "pt_emb":nlp.pt_emb,"reg_lamda":0.1,"supervise_mode":True,"reg_lamda":0.0}
ptIf_para={"class":"PyTrain_Interface_sup","version":2}
pt1=PyTrain_Custom(dataset_sup, [lsize_in,lsize_out], bigru_coop_pos, 20000, ptIf_para, learning_rate=1.0e-3, batch=batch, window=window_size, para=pt_para)

Using adam optimizer
cuda:2


In [83]:
pt1.run_training(step=20000,lr=1.0e-3)

Using adam optimizer
Perlexity:  0 9.709436332011194
Perlexity:  200 8.4310678544617
Perlexity:  400 9.238086627605265
Perlexity:  600 8.872411953631282
Perlexity:  800 8.305445726528848
Perlexity:  1000 9.388313897836166
Perlexity:  1200 9.167401682467412
Perlexity:  1400 8.847604055580463
Perlexity:  1600 9.233297363488935
Perlexity:  1800 8.795205414145205
Perlexity:  2000 9.404031292559184
Perlexity:  2200 9.140461325039114
Perlexity:  2400 9.333520724825858
Perlexity:  2600 8.91759409397179
Perlexity:  2800 9.579029869738786
Perlexity:  3000 8.663232287936632
Perlexity:  3200 8.933241166531873
Perlexity:  3400 9.893747767022735
Perlexity:  3600 9.272496830817897
Perlexity:  3800 8.680345108505223
Perlexity:  4000 9.293041815743269
Perlexity:  4200 8.73565352724773
Perlexity:  4400 8.954234174415763
Perlexity:  4600 9.115421169121603
Perlexity:  4800 8.572787556979222
Perlexity:  5000 8.893845120417158
Perlexity:  5200 8.805664876319343
Perlexity:  5400 8.985423103274947
Perlexity:

In [82]:
2**3.684257

12.854993717639495

## Combinatorial Test

In [84]:
cdbook=load_data("cdbook_1.data")

Data load from  cdbook_1.data


In [101]:
rstart=int(np.random.rand()*len(datasetc))
testd=datasetc[rstart:rstart+50]
testp=pos_onlyc[rstart:rstart+50]
print(testd)

[107, 18, 4, 0, 10, 7, 7, 18, 178, 1, 3, 7, 7, 3839, 10703, 33, 19, 6639, 6, 154, 463, 9564, 4254, 103, 24, 0, 545, 3825, 33, 715, 6, 1409, 622, 578, 3216, 7589, 4, 0, 194, 74, 282, 159, 1625, 16, 59, 19, 99, 1094, 56, 444]


In [102]:
testt0=""
for item in testd:
    testt0=testt0+nlp.id_to_word[item]+" "
print(testt0)
testt=[(ii,nlp.id_to_word[item]) for ii,item in enumerate(testd)]
print(testt)

number three in the two zero zero three list of one zero zero worst britons which was confined to those living narrowly missing out on the top spot which went to labour prime minister tony blair in the end however few could argue that there was any woman who played 
[(0, 'number'), (1, 'three'), (2, 'in'), (3, 'the'), (4, 'two'), (5, 'zero'), (6, 'zero'), (7, 'three'), (8, 'list'), (9, 'of'), (10, 'one'), (11, 'zero'), (12, 'zero'), (13, 'worst'), (14, 'britons'), (15, 'which'), (16, 'was'), (17, 'confined'), (18, 'to'), (19, 'those'), (20, 'living'), (21, 'narrowly'), (22, 'missing'), (23, 'out'), (24, 'on'), (25, 'the'), (26, 'top'), (27, 'spot'), (28, 'which'), (29, 'went'), (30, 'to'), (31, 'labour'), (32, 'prime'), (33, 'minister'), (34, 'tony'), (35, 'blair'), (36, 'in'), (37, 'the'), (38, 'end'), (39, 'however'), (40, 'few'), (41, 'could'), (42, 'argue'), (43, 'that'), (44, 'there'), (45, 'was'), (46, 'any'), (47, 'woman'), (48, 'who'), (49, 'played')]


In [104]:
testt=[(ii,nlp_pos.id_to_word[item]) for ii,item in enumerate(testp)]
print(testt)
wrdp=22
print(nlp.id_to_word[testd[wrdp]],testt[wrdp])

[(0, 'NN'), (1, 'CD'), (2, 'IN'), (3, 'DT'), (4, 'CD'), (5, 'NN'), (6, 'CD'), (7, 'CD'), (8, 'NN'), (9, 'IN'), (10, 'CD'), (11, 'NN'), (12, 'CD'), (13, 'JJS'), (14, 'NNS'), (15, 'WDT'), (16, 'VBD'), (17, 'VBN'), (18, 'TO'), (19, 'DT'), (20, 'VBG'), (21, 'RB'), (22, 'VBG'), (23, 'RP'), (24, 'IN'), (25, 'DT'), (26, 'JJ'), (27, 'NN'), (28, 'WDT'), (29, 'VBD'), (30, 'TO'), (31, 'VB'), (32, 'JJ'), (33, 'NN'), (34, 'NN'), (35, 'NN'), (36, 'IN'), (37, 'DT'), (38, 'NN'), (39, 'RB'), (40, 'JJ'), (41, 'MD'), (42, 'VB'), (43, 'IN'), (44, 'EX'), (45, 'VBD'), (46, 'DT'), (47, 'NN'), (48, 'WP'), (49, 'VBD')]
missing (22, 'VBG')


In [105]:
inputd=ptT.pt_emb(torch.LongTensor(testd)).view(50,1,300).to(cuda_device)
hd=bigru_pos.initHidden_cuda(cuda_device,1)
bigru_pos.coop_mode=False
outputp,hn=bigru_pos.forward(inputd,hd,schedule=1.0)
outputp=outputp.cpu().detach().numpy()

In [106]:
ids,vals=sort_w_arg(outputp[wrdp][0])
for ii in range(10):
    print(nlp_pos.id_to_word[ids[ii]])

VBG
NN
VBD
IN
VBZ
WDT
WRB
NNS
VBP
FW


In [107]:
bigru_pos.coop_mode=True
inputd=ptT.pt_emb(torch.LongTensor(testd)).view(50,1,300).to(cuda_device)
hd=bigru_coop.initHidden_cuda(cuda_device,1)
outputd,hn=bigru_coop.forward(inputd,hd,schedule=1.0)
outputd=outputd.cpu().detach().numpy()

In [108]:
pos_context=bigru_pos.context
print(pos_context[wrdp,0,:])

tensor([0.2000, 0.0000, 0.0000, 0.4000, 0.0000], device='cuda:2',
       grad_fn=<SliceBackward>)


In [115]:
pos_context=bigru_pos.context
pos_context[wrdp,0,:]=torch.Tensor(cdbook["VBZ"])
outputrp=bigru_coop.forward_comb(pos_context)
outputrp=outputrp.cpu().detach().numpy()

ids,vals=sort_w_arg(outputrp[wrdp][0])
for ii in range(10):
    print(nlp.id_to_word[ids[ii]])

UNK
comes
refers
leads
means
is
goes
appears
continues
links


In [96]:
print(cdbook["VBZ"])
print(cdbook["VBG"])
print(cdbook["VBP"])
print(cdbook["VBD"])
print(cdbook["VBN"])
print(cdbook["VB"])

[0.01484927 0.00764513 0.         0.48974793 0.20479242]
[0.19654762 0.00582862 0.         0.46662935 0.01994436]
[0.0521721  0.02013154 0.         0.1821156  0.20654435]
[0.02024911 0.00471298 0.         0.1721125  0.02215512]
[0.02438429 0.00452785 0.         0.01642115 0.01744261]
[0.01820732 0.004809   0.         0.02950219 0.24843971]


In [8]:
2**(10.2-3.2)

127.99999999999991

## Combinatorial Test with Morphy Data